In [3]:
!touch /tmp/tmp.txt
!touch /tmp/tmp0.txt
!touch /tmp/tmp1.txt
!touch /tmp/tmp2.txt
!touch /tmp/tmp3.txt
!touch /tmp/tmp4.txt

In [2]:
from taigapy import default_tc as tc

new_dataset_id = tc.create_dataset(
    "CN latest",
    dataset_description="copy number data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

Uploading temp to S3
Finished uploading temp to S3
Uploading temp to Taiga
Finished uploading temp to Taiga
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/b524d6547a0b4957af5e9a05d601dfa8



In [4]:
new_dataset_id = tc.create_dataset(
    "expression",
    dataset_description="expression data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp0.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

Uploading temp to S3
Finished uploading temp to S3
Uploading temp to Taiga
Finished uploading temp to Taiga
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/9c3d558b0b5842aa8cf69b11df263de0



In [5]:
new_dataset_id = tc.create_dataset(
    "mutations latest",
    dataset_description="mutations latest data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp1.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

Uploading temp to S3
Finished uploading temp to S3
Uploading temp to Taiga
Finished uploading temp to Taiga
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/381ee95950384e8997128e4897dc1da9



In [6]:
new_dataset_id = tc.create_dataset(
    "fusions",
    dataset_description="fusions data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp2.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

Uploading temp to S3
Finished uploading temp to S3
Uploading temp to Taiga
Finished uploading temp to Taiga
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/301aba3dd9be4052acdf1de2a2fafd77



In [7]:
new_dataset_id = tc.create_dataset(
    "CN Achilles version",
    dataset_description="CN Achilles version data", # optional (but recommended)
    upload_files=[
        {
            "path": "/tmp/tmp3.txt",
            "name": "temp", # optional, will use file name if not provided
            "format": "Raw", # or "NumericMatrixCSV" or "TableCSV"
            "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
        }
    ],
    folder_id="af8daf85d5f84f29b9f1494e8f05b9fd", # optional, will default to your home folder if not provided
)

Uploading temp to S3
Finished uploading temp to S3
Uploading temp to Taiga
Finished uploading temp to Taiga
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/8f67bd9591c3471d91ca1a3c8cf90855

